In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torcheval.metrics import BinaryAccuracy
from model import Net

if torch.cuda.is_available():
    torch.set_default_device('cuda')
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    torch.set_default_device('mps')
    device = torch.device('mps')
else:
    torch.set_default_device('cpu')
    device = torch.device('cpu')

torch.manual_seed(0)
batch_size = 150

Using cache found in /Users/kiran/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/kiran/anaconda3/envs/5523Final/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kiran/anaconda3/envs/5523Final/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((256, 256)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [3]:
train = datasets.OxfordIIITPet("", split="trainval", transform=transform, target_types="binary-category", download=True)
test = datasets.OxfordIIITPet("", split="test", transform=transform, target_types="binary-category", download=True)

In [4]:
net = Net().to(device)

In [5]:
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
metric = BinaryAccuracy()

In [12]:
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device).manual_seed(0))

In [13]:
batches = train.__len__() // batch_size
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device).to(torch.float32)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        metric.update(outputs, labels)
        if i % 10 == 9:
            print('[%d, %5d/%d] loss: %.3f, acc:%.3f' %
                (epoch + 1, i + 1, batches, running_loss / 10, metric.compute()))
            running_loss = 0.0
            metric.reset()
        elif i == batches - 1:
            print('[%d, %5d/%d] loss: %.3f, acc:%.3f' %
                (epoch + 1, i + 1, batches, running_loss / (batches % 10), metric.compute()))
            running_loss = 0.0
            metric.reset()

print('Finished Training')

[1,    10/24] loss: 0.092, acc:0.744
[1,    20/24] loss: 0.066, acc:0.975
[1,    24/24] loss: 0.057, acc:0.980
Finished Training


In [15]:
torch.save(net, "model.pth")

In [14]:
testloader = torch.utils.data.DataLoader(test, batch_size=3, num_workers=2)
metric.reset()
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    metric.update(outputs, labels)
print(f"Test accuracy: {metric.compute()}")

Test accuracy: 0.6593077182769775


Test accuracy: 0.6544017195701599